In [1]:
import seaborn as sns

import ipywidgets as widgets
from ipywidgets import interact

import matplotlib.pyplot as plt


In [2]:
from japr_data.kungsbacka import kungsbacka as data

In [3]:
# data = pd.read_csv("data.csv", parse_dates=True, index_col=0)

In [4]:
features = ["temperature", "sunshine", "precipitation"]
feature_selector = widgets.SelectMultiple(
    options=features,
    value=features,
    description='Variables:',
)

freqs = ["D", "W", "2W", "ME"]
freq_selector = widgets.RadioButtons(
    options=freqs,
    description='Temporal resolution:',
    disabled=False,
    orientation='horizontal',
)

display(feature_selector)

SelectMultiple(description='Variables:', index=(0, 1, 2), options=('temperature', 'sunshine', 'precipitation')…

In [ ]:
def select_and_resample(freq, features):
    resample_dict = {"real_consumption":"sum", "sunshine": "mean", "temperature": "mean", "precipitation": "sum"}
    vars = ["real_consumption"] + features
    return data.loc[:, vars].resample(freq).agg({k:v for k, v in resample_dict.items() if k in vars})

hue_options = set(features).difference(set(feature_selector.value))
hue_picker = widgets.ToggleButtons(
    options=hue_options,
    description='Hue:',
)


@interact(freq = freq_selector, hue = hue_picker)
def make_pairplot(freq, hue):
    if hue is not None:
        additional_features = list(feature_selector.value) + [hue]
    else:
        additional_features = list(feature_selector.value)
    resampled_data = select_and_resample(freq, additional_features)
    sns.pairplot(resampled_data, hue=hue)

interactive(children=(RadioButtons(description='Temporal resolution:', options=('D', 'W', '2W', 'ME'), orienta…

In [8]:
features = list(feature_selector.value) + [hue_picker.value]
selected_data = select_and_resample(freq_selector.value, features)
selected_data.describe()

,real_consumption,sunshine,temperature,precipitation
count,210.000000,210.000000,210.000000,210.000000
mean,104269.199573,5.112630,9.598096,14.995238
std,11226.240640,3.791136,6.889655,16.558609
min,77255.000000,0.000000,-7.133333,0.000000
25%,96856.704459,1.905714,4.043750,2.425000
50%,101247.549252,4.237996,9.431250,9.050000
75%,108624.166546,7.813423,15.905655,22.975000
max,145413.000000,15.726905,23.216667,99.200000


In [ ]:
datplot = selected_data.loc[selected_data["precipitation"] < 40, :]

temp_selector = widgets.FloatRangeSlider(min=-7, max=23, readout_format='.1f')

@interact()
def plot_reg_by_temperature(temp_cutoff = temp_selector):
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    sns.scatterplot(datplot, x="precipitation", y="real_consumption", ax=ax, alpha=0.3)
    sns.regplot(datplot.loc[datplot["temperature"].between(*temp_cutoff),:], x="precipitation", y="real_consumption", ax=ax, scatter_kws=dict(alpha=0))

interactive(children=(FloatRangeSlider(value=(0.5, 15.5), description='temp_cutoff', max=23.0, min=-7.0, reado…